In [1]:
import os
import sys
sys.path.append('../utils')
from spot import DailyDataset, TOTAL_STD, TOTAL_MEAN, get_daily_spot, reduce_hour_mean, get_loader
from tool import to_gpu

import numpy as np
import pandas as pd
from pandas.tseries.offsets import Day


import matplotlib.pyplot as plt

import types

%matplotlib inline

In [2]:
path = './seasonal'
TOTAL_STD = 22.732
TOTAL_MEAN = 38.99

In [3]:
day = pd.read_csv(os.path.join(path, 'day.csv'),
                   index_col='datetime', parse_dates=True)
day

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
datetime,,,,,,,,,,,,,,,,,,,,,
2011-06-17,-0.036482,0.012318,-0.125990,-0.150465,-0.234397,-0.131304,-0.112682,-0.183056,-0.197364,-0.250659,...,0.054575,-0.068413,-0.008661,-0.077514,-0.065859,-0.278228,-0.384493,-0.152602,0.014931,0.085059
2011-06-18,-0.006104,-0.113808,-0.171062,-0.210322,-0.235842,-0.134399,-0.101393,-0.147800,-0.222631,-0.263560,...,-0.051866,-0.154214,-0.117447,-0.252438,-0.248523,-0.505622,-0.492286,-0.303623,-0.073111,-0.040284
2011-06-19,-0.259406,-0.202590,-0.356893,-0.289491,-0.331092,-0.311227,-0.278660,-0.523616,-0.500148,-0.517993,...,-0.307826,-0.390568,-0.369900,-0.536536,-0.436639,-0.658520,-0.743153,-0.527953,-0.216902,-0.254521
2011-06-20,-0.191012,-0.253262,-0.284653,-0.168199,-0.207732,-0.156264,-0.118625,-0.341057,-0.301769,-0.382292,...,-0.156078,-0.245258,-0.288354,-0.377439,-0.335257,-0.728889,-0.769335,-0.472924,-0.230575,-0.147718
2011-06-21,-0.081106,-0.160148,-0.232892,-0.324321,-0.279837,-0.204294,-0.147624,-0.091927,-0.214051,-0.299344,...,-0.033527,-0.054088,0.054906,-0.088216,-0.234547,-0.641611,-0.531123,-0.353015,-0.063107,-0.055809
2011-06-22,-0.094151,-0.118533,-0.264692,-0.388567,-0.345382,-0.238092,-0.098099,-0.084702,-0.137319,-0.194657,...,0.213198,0.131904,0.153630,0.015544,-0.192186,-0.636765,-0.446887,-0.264243,0.061804,0.001798
2011-06-23,-0.131405,-0.221911,-0.392320,-0.388525,-0.327467,-0.192178,-0.132972,-0.225342,-0.359953,-0.389700,...,-0.002979,-0.069143,-0.046967,-0.179825,-0.336757,-0.617690,-0.559608,-0.401742,-0.088489,-0.093421
2011-06-24,-0.165703,-0.142521,-0.314363,-0.314898,-0.285362,-0.144048,-0.106112,-0.128866,-0.276606,-0.302318,...,-0.017192,-0.040978,-0.056331,-0.174335,-0.280352,-0.637484,-0.536454,-0.387312,-0.056169,-0.072692
2011-06-25,-0.134960,-0.104131,-0.246338,-0.304748,-0.230523,-0.167783,-0.098306,-0.009190,-0.157260,-0.165676,...,0.044270,0.073521,0.034961,-0.107343,-0.175044,-0.575461,-0.504498,-0.270822,-0.055818,-0.027428


In [4]:
week = pd.read_csv(os.path.join(path, 'week.csv'),
                   index_col='datetime', parse_dates=True)
week

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
datetime,,,,,,,,,,,,,,,,,,,,,
2011-06-17,0.015661,0.007380,0.142045,0.040514,0.089489,0.153715,0.246973,0.250023,0.219512,0.207507,...,0.233605,0.192930,0.161556,0.116079,0.157642,0.057223,0.058763,0.119093,0.073283,0.102476
2011-06-18,0.011621,0.059583,0.183548,0.078545,0.101338,0.080095,0.062659,-0.011114,0.036854,0.125127,...,0.102166,0.005745,0.026878,-0.009298,0.030994,-0.075568,0.011191,0.000222,0.088686,0.105465
2011-06-19,-0.217096,-0.110150,-0.001721,-0.112699,0.053799,0.229197,-0.306310,-0.626898,-0.613187,-0.551589,...,-0.334134,-0.392386,-0.434983,-0.341952,-0.421904,-0.601613,-0.579724,-0.453098,-0.194307,-0.156618
2011-06-20,-0.240654,-0.274188,-0.066168,-0.118321,0.004018,0.072683,0.217081,0.409524,0.302029,0.297165,...,0.329149,0.402779,0.283521,0.275519,0.449190,0.259216,0.177737,0.138502,0.090953,0.150783
2011-06-21,-0.099073,-0.107331,-0.027099,-0.057879,-0.059537,-0.027135,0.043368,0.105075,0.003376,-0.004886,...,0.099815,0.106142,0.072186,0.081983,0.141046,0.055466,-0.047699,-0.013335,-0.034846,-0.015692
2011-06-22,-0.074264,-0.110928,-0.019120,-0.067842,-0.084512,-0.025823,0.064761,0.144940,0.004246,0.092848,...,0.062147,0.028731,0.079658,-0.000086,0.056070,0.003915,-0.040589,-0.022325,-0.004634,0.046511
2011-06-23,-0.020434,-0.096924,-0.031268,-0.085718,-0.040464,0.040160,0.019721,0.007591,-0.109916,-0.041616,...,0.075865,0.061149,0.088505,0.047312,0.024292,-0.017927,-0.102258,0.011544,-0.011694,0.039514
2011-06-24,-0.064614,-0.056863,0.059105,-0.022968,0.074449,0.048370,0.198043,0.205738,0.143545,0.119864,...,0.124943,0.101608,0.089569,0.044791,0.014605,-0.009047,0.027212,0.085234,0.065428,0.078244
2011-06-25,-0.059491,0.042259,0.111137,0.030138,0.065485,0.004292,0.046342,0.030648,0.006935,0.062524,...,0.067925,-0.003671,0.044922,0.010653,0.051811,-0.084374,0.072374,0.048704,0.091667,0.090284


In [5]:
start_date = '2012-01-01'
end_date = '2016-06-30'
dt = pd.to_datetime(end_date)

In [6]:
t=week.loc[pd.to_datetime(start_date):pd.to_datetime(end_date)].values

In [7]:
t

array([[-0.11207492, -0.01224375,  0.02455396, ..., -0.3092294 ,
        -0.14997037, -0.14556114],
       [-0.10354   , -0.0922337 , -0.05790211, ..., -0.04224164,
        -0.03348773, -0.02534516],
       [-0.05742504, -0.08122887, -0.10049006, ..., -0.0150588 ,
        -0.0337239 , -0.0241482 ],
       ...,
       [-0.09038021, -0.0366924 , -0.00291857, ...,  0.03053609,
         0.03308648,  0.04456644],
       [-0.07676131, -0.01890462,  0.02687765, ..., -0.02057035,
         0.01543204,  0.03524301],
       [-0.06909911, -0.03846499,  0.02680144, ..., -0.12200463,
        -0.0390377 , -0.01389275]])

In [8]:
t.shape

(1643, 24)

In [9]:
spot = get_daily_spot(reduce_hour_mean())

In [10]:
s=spot.loc[pd.to_datetime(start_date):pd.to_datetime(end_date)].values

In [11]:
s

array([[-0.87485621, -1.20101696, -1.3069537 , ..., -0.4925009 ,
        -0.91187946, -0.98344276],
       [-0.36772969, -0.54546531, -0.6338497 , ..., -0.60133426,
        -0.37109994, -0.38538716],
       [-0.72022837, -0.72178064, -0.64141613, ..., -0.50745779,
        -0.62686275, -0.64326152],
       ...,
       [-0.44282207, -0.55201995, -0.55361039, ..., -0.53455615,
        -0.61137797, -0.50099511],
       [-0.37947525, -0.3166689 , -0.21356112, ..., -0.63309566,
        -0.56166831, -0.44820609],
       [-0.32052751, -0.23748537, -0.19728451, ..., -0.45537262,
        -0.37690674, -0.38881844]])

In [12]:
s.shape

(1643, 24)

In [13]:
p = np.concatenate((t,s), axis=-1)

In [14]:
p

array([[-0.11207492, -0.01224375,  0.02455396, ..., -0.4925009 ,
        -0.91187946, -0.98344276],
       [-0.10354   , -0.0922337 , -0.05790211, ..., -0.60133426,
        -0.37109994, -0.38538716],
       [-0.05742504, -0.08122887, -0.10049006, ..., -0.50745779,
        -0.62686275, -0.64326152],
       ...,
       [-0.09038021, -0.0366924 , -0.00291857, ..., -0.53455615,
        -0.61137797, -0.50099511],
       [-0.07676131, -0.01890462,  0.02687765, ..., -0.63309566,
        -0.56166831, -0.44820609],
       [-0.06909911, -0.03846499,  0.02680144, ..., -0.45537262,
        -0.37690674, -0.38881844]])

In [15]:
p[:,0:24]=s

In [16]:
p[0]

array([-0.87485621, -1.20101696, -1.3069537 , -1.21273237, -1.22340101,
       -1.03208887, -0.84001979, -0.81175228, -0.82459045, -0.7302771 ,
       -0.54287241, -0.27251091, -0.40464193, -0.44660119, -0.37655026,
       -0.46506768, -0.28862231, -0.26700001, -0.29048471, -0.47273251,
       -0.57485289, -0.4925009 , -0.91187946, -0.98344276, -0.87485621,
       -1.20101696, -1.3069537 , -1.21273237, -1.22340101, -1.03208887,
       -0.84001979, -0.81175228, -0.82459045, -0.7302771 , -0.54287241,
       -0.27251091, -0.40464193, -0.44660119, -0.37655026, -0.46506768,
       -0.28862231, -0.26700001, -0.29048471, -0.47273251, -0.57485289,
       -0.4925009 , -0.91187946, -0.98344276])

In [17]:
dataset = DailyDataset(N=2000, W=14)
loader = get_loader(dataset, batch_size=64, shuffle=True, num_workers=2)

Data build range: [window(2010-06-27 00:00:00) - 2010-07-11 00:00:00, 2015-12-31 00:00:00]


In [19]:
trainX, trainY = dataset.get_io('2012-01-01', '2015-12-31')
testX, testY = dataset.get_io('2012-01-01', '2016-08-30')

In [20]:
trainX.shape

torch.Size([1474, 24])

In [21]:
trainY.shape

torch.Size([1474, 24])

In [22]:
testX.shape

torch.Size([1716, 24])

In [23]:
testY.shape

torch.Size([1716, 24])

In [24]:
enumerate(loader)

In [25]:
spot=get_daily_spot(reduce_hour_mean())

In [26]:
end_date='2015-12-31'
dt = pd.to_datetime(end_date)

In [27]:
sliding = spot.loc[pd.to_datetime(dt) - Day(14+1):pd.to_datetime(dt)].values

In [28]:
sliding

array([[-0.36451836, -0.26651933, -0.22543865, -0.25742708, -0.36584338,
        -0.36144836, -0.22621546, -0.29930287, -0.34575004, -0.39722238,
        -0.27813548, -0.3802445 , -0.30984165, -0.23953626, -0.15813569,
        -0.05749246, -0.22210815, -0.57775137, -0.54347609, -0.36645061,
        -0.58549867, -0.34319596,  0.14174538, -0.45876389],
       [-0.15644164, -0.06372151, -0.12293997, -0.15448849, -0.1700841 ,
        -0.13885466, -0.11843788, -0.3265772 , -0.39765925, -0.38930403,
        -0.40394931, -0.19416321, -0.18270809, -0.23029818, -0.18277057,
        -0.06453099, -0.19571364, -0.48537059, -0.39830628, -0.44475433,
        -0.57758032, -0.48088732, -0.09404557, -0.20933577],
       [-0.1907545 , -0.176778  , -0.10974272, -0.2847014 , -0.56160267,
        -0.77100318, -0.83856809, -0.77528387, -0.56394466, -0.59298166,
        -0.55923701, -0.45414913, -0.50956011, -0.46696895, -0.33365918,
        -0.32935591, -0.31140957, -0.63273994, -0.85097213, -0.73905311,
  

In [29]:
p=sliding[13:15].reshape(24*2)
p

array([-0.09617418, -0.16534038, -0.29318456, -0.44614782, -0.48329895,
       -0.69137974, -0.72463179, -0.49638188, -0.57318274, -0.50060088,
       -0.53636176, -0.55972717, -0.61249869, -0.56066946, -0.49554551,
       -0.43185459, -0.31624856, -0.6309803 , -0.55227426, -0.38316714,
       -0.74914463, -0.3607923 , -0.35139204, -0.18778026, -0.79166951,
       -0.68795167, -0.68030404, -0.94324443, -1.11544747, -1.07717949,
       -1.1649802 , -1.19055749, -1.10855138, -1.05312595, -0.9313996 ,
       -0.76868371, -0.62437622, -0.55890983, -0.43087896, -0.33551463,
       -0.28237561, -0.56719357, -0.64641468, -0.73905311, -0.75574326,
       -0.53323643, -0.21194105, -0.83884484])

In [30]:
a=[]
a += [p]
a += [p]

In [31]:
a

[array([-0.09617418, -0.16534038, -0.29318456, -0.44614782, -0.48329895,
        -0.69137974, -0.72463179, -0.49638188, -0.57318274, -0.50060088,
        -0.53636176, -0.55972717, -0.61249869, -0.56066946, -0.49554551,
        -0.43185459, -0.31624856, -0.6309803 , -0.55227426, -0.38316714,
        -0.74914463, -0.3607923 , -0.35139204, -0.18778026, -0.79166951,
        -0.68795167, -0.68030404, -0.94324443, -1.11544747, -1.07717949,
        -1.1649802 , -1.19055749, -1.10855138, -1.05312595, -0.9313996 ,
        -0.76868371, -0.62437622, -0.55890983, -0.43087896, -0.33551463,
        -0.28237561, -0.56719357, -0.64641468, -0.73905311, -0.75574326,
        -0.53323643, -0.21194105, -0.83884484]),
 array([-0.09617418, -0.16534038, -0.29318456, -0.44614782, -0.48329895,
        -0.69137974, -0.72463179, -0.49638188, -0.57318274, -0.50060088,
        -0.53636176, -0.55972717, -0.61249869, -0.56066946, -0.49554551,
        -0.43185459, -0.31624856, -0.6309803 , -0.55227426, -0.38316714,
  

In [32]:
b=np.array(a)

In [33]:
b.shape

(2, 48)

In [34]:
tmp=[]
idx=0
for i in range(14):
    tmp += [sliding[i:i+2].reshape(24*2)]


In [35]:
def _sliding_window(date):
    _sliding = spot.loc[pd.to_datetime(date) - Day(15):
                             pd.to_datetime(date)].values
    x = _sliding[:15]
    y = _sliding[-14:]
    return x, y

In [36]:
end_date='2015-12-31'
dt = pd.to_datetime(end_date)
X_ = []
Y_ = []
dates_ = []
for _ in range(2):
    x, y = _sliding_window(dt)
    X_.append(x)
    Y_.append(y)
    dates_.append(dt)
    dt -= Day(1)

In [37]:
sliding.shape

(16, 24)

In [38]:
X_

[array([[-0.36451836, -0.26651933, -0.22543865, -0.25742708, -0.36584338,
         -0.36144836, -0.22621546, -0.29930287, -0.34575004, -0.39722238,
         -0.27813548, -0.3802445 , -0.30984165, -0.23953626, -0.15813569,
         -0.05749246, -0.22210815, -0.57775137, -0.54347609, -0.36645061,
         -0.58549867, -0.34319596,  0.14174538, -0.45876389],
        [-0.15644164, -0.06372151, -0.12293997, -0.15448849, -0.1700841 ,
         -0.13885466, -0.11843788, -0.3265772 , -0.39765925, -0.38930403,
         -0.40394931, -0.19416321, -0.18270809, -0.23029818, -0.18277057,
         -0.06453099, -0.19571364, -0.48537059, -0.39830628, -0.44475433,
         -0.57758032, -0.48088732, -0.09404557, -0.20933577],
        [-0.1907545 , -0.176778  , -0.10974272, -0.2847014 , -0.56160267,
         -0.77100318, -0.83856809, -0.77528387, -0.56394466, -0.59298166,
         -0.55923701, -0.45414913, -0.50956011, -0.46696895, -0.33365918,
         -0.32935591, -0.31140957, -0.63273994, -0.85097213, -

In [39]:
x = sliding[:15]
y = sliding[-14:]

In [40]:
Y_

[array([[-0.1907545 , -0.176778  , -0.10974272, -0.2847014 , -0.56160267,
         -0.77100318, -0.83856809, -0.77528387, -0.56394466, -0.59298166,
         -0.55923701, -0.45414913, -0.50956011, -0.46696895, -0.33365918,
         -0.32935591, -0.31140957, -0.63273994, -0.85097213, -0.73905311,
         -0.82964789, -0.58470572,  0.03132835, -0.18822017],
        [-0.64869925, -0.57269565, -0.58572371, -0.59747635, -1.05649973,
         -1.30461219, -1.51734691, -1.40875211, -1.39097264, -1.34786465,
         -1.20634241, -1.10037471, -0.92263419, -0.93151233, -0.93061502,
         -0.70063868, -0.69412996, -0.81882123, -0.7150404 , -0.55693099,
         -0.82040981, -0.56930892, -0.46576825, -0.583258  ],
        [-0.56643636, -0.38573454, -0.52765579, -0.5587644 , -0.62318986,
         -0.5290535 , -0.18398425, -0.16821014, -0.19134216, -0.34047418,
         -0.30101072, -0.36748715, -0.16027276, -0.39218451, -0.37193122,
         -0.22729714, -0.2454233 , -0.51836373, -0.51884121, -

In [41]:
x

array([[-0.36451836, -0.26651933, -0.22543865, -0.25742708, -0.36584338,
        -0.36144836, -0.22621546, -0.29930287, -0.34575004, -0.39722238,
        -0.27813548, -0.3802445 , -0.30984165, -0.23953626, -0.15813569,
        -0.05749246, -0.22210815, -0.57775137, -0.54347609, -0.36645061,
        -0.58549867, -0.34319596,  0.14174538, -0.45876389],
       [-0.15644164, -0.06372151, -0.12293997, -0.15448849, -0.1700841 ,
        -0.13885466, -0.11843788, -0.3265772 , -0.39765925, -0.38930403,
        -0.40394931, -0.19416321, -0.18270809, -0.23029818, -0.18277057,
        -0.06453099, -0.19571364, -0.48537059, -0.39830628, -0.44475433,
        -0.57758032, -0.48088732, -0.09404557, -0.20933577],
       [-0.1907545 , -0.176778  , -0.10974272, -0.2847014 , -0.56160267,
        -0.77100318, -0.83856809, -0.77528387, -0.56394466, -0.59298166,
        -0.55923701, -0.45414913, -0.50956011, -0.46696895, -0.33365918,
        -0.32935591, -0.31140957, -0.63273994, -0.85097213, -0.73905311,
  

In [42]:
y

array([[-0.1907545 , -0.176778  , -0.10974272, -0.2847014 , -0.56160267,
        -0.77100318, -0.83856809, -0.77528387, -0.56394466, -0.59298166,
        -0.55923701, -0.45414913, -0.50956011, -0.46696895, -0.33365918,
        -0.32935591, -0.31140957, -0.63273994, -0.85097213, -0.73905311,
        -0.82964789, -0.58470572,  0.03132835, -0.18822017],
       [-0.64869925, -0.57269565, -0.58572371, -0.59747635, -1.05649973,
        -1.30461219, -1.51734691, -1.40875211, -1.39097264, -1.34786465,
        -1.20634241, -1.10037471, -0.92263419, -0.93151233, -0.93061502,
        -0.70063868, -0.69412996, -0.81882123, -0.7150404 , -0.55693099,
        -0.82040981, -0.56930892, -0.46576825, -0.583258  ],
       [-0.56643636, -0.38573454, -0.52765579, -0.5587644 , -0.62318986,
        -0.5290535 , -0.18398425, -0.16821014, -0.19134216, -0.34047418,
        -0.30101072, -0.36748715, -0.16027276, -0.39218451, -0.37193122,
        -0.22729714, -0.2454233 , -0.51836373, -0.51884121, -0.4069222 ,
  

In [43]:
start_date = dates[-1]

for _ in range(15):
    dates_.append(dt)
    dt -= Day(1)

X = np.array(list(reversed(X_)))
Y = np.array(list(reversed(Y_)))
dates = np.array(list(reversed(dates_)))

NameError: name 'dates' is not defined

In [44]:
X.shape

NameError: name 'X' is not defined

In [32]:
Y.shape

(2, 14, 24)

In [33]:
dates

array([Timestamp('2015-12-15 00:00:00'), Timestamp('2015-12-16 00:00:00'),
       Timestamp('2015-12-17 00:00:00'), Timestamp('2015-12-18 00:00:00'),
       Timestamp('2015-12-19 00:00:00'), Timestamp('2015-12-20 00:00:00'),
       Timestamp('2015-12-21 00:00:00'), Timestamp('2015-12-22 00:00:00'),
       Timestamp('2015-12-23 00:00:00'), Timestamp('2015-12-24 00:00:00'),
       Timestamp('2015-12-25 00:00:00'), Timestamp('2015-12-26 00:00:00'),
       Timestamp('2015-12-27 00:00:00'), Timestamp('2015-12-28 00:00:00'),
       Timestamp('2015-12-29 00:00:00'), Timestamp('2015-12-30 00:00:00'),
       Timestamp('2015-12-31 00:00:00')], dtype=object)

In [ ]:
def get_io(self, start_date, end_date):
    _sliding = self.spot.loc[pd.to_datetime(start_date) - Day(self.W):
                             pd.to_datetime(end_date)].values
    i_stream = _sliding[:-1]
    o_stream = _sliding[1:]
    return torch.from_numpy(i_stream).float(), torch.from_numpy(o_stream).float()

In [6]:
dataset = PeriodDataset(N=2000, W=14, P=7)
loader = get_loader(dataset, batch_size=64, shuffle=True, num_workers=2)

Data build range: [window(2010-06-21 00:00:00) - 2010-07-11 00:00:00, 2015-12-31 00:00:00]


In [7]:
trainX, trainY = dataset.get_io('2012-01-01', '2015-12-31')
testX, testY = dataset.get_io('2012-01-01', '2016-08-30')

(1481, 24)
(1724, 24)


In [8]:
trainX.shape

torch.Size([1473, 168])

In [9]:
trainY.shape

torch.Size([1473, 24])

In [ ]:
# test self decomposion

In [6]:
dataset = DailyDataset(N=200, W=14)
loader = get_loader(dataset, batch_size=10, shuffle=True, num_workers=2)

Data build range: [window(2015-06-01 00:00:00) - 2015-06-15 00:00:00, 2015-12-31 00:00:00]


In [8]:
for _, (x, y, item) in enumerate(loader):
    print('x\' shape:', x.shape)
    print('y\' shape:', y.shape)
    break

x' shape: torch.Size([10, 14, 24])
y' shape: torch.Size([10, 14, 24])


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f1c8413add8>>
Traceback (most recent call last):
  File "/home/xurenchao/install/anaconda3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/xurenchao/install/anaconda3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/xurenchao/install/anaconda3/lib/python3.5/multiprocessing/queues.py", line 337, in get
    return ForkingPickler.loads(res)
  File "/home/xurenchao/install/anaconda3/lib/python3.5/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/xurenchao/install/anaconda3/lib/python3.5/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/home/xurenchao/install/anaconda3/lib/python3.5/mu

In [9]:
x

tensor([[[-0.2783, -0.3519, -0.0495,  ...,  0.2362, -0.2938, -0.3413],
         [-0.1274, -0.0532,  0.0979,  ..., -0.2319, -0.2353, -0.1108],
         [ 0.0930,  0.1171,  0.1863,  ..., -0.0397, -0.1367, -0.0668],
         ...,
         [-0.0561,  0.0278, -0.0226,  ..., -0.2209, -0.1913, -0.1662],
         [-0.4358, -0.4557, -0.2439,  ...,  0.5067, -0.3822, -0.3950],
         [ 0.3605,  0.3348,  0.4683,  ..., -0.1664,  0.6050,  0.3212]],

        [[ 0.0129,  0.2103,  0.2228,  ..., -0.1034, -0.0391, -0.0778],
         [-0.0561,  0.0278, -0.0226,  ..., -0.2209, -0.1913, -0.1662],
         [-0.4358, -0.4557, -0.2439,  ...,  0.5067, -0.3822, -0.3950],
         ...,
         [-0.0575, -0.0774,  0.1529,  ...,  0.1152,  0.2258,  0.0225],
         [ 0.1893,  0.2064,  0.3742,  ..., -0.0960,  0.2157,  0.1479],
         [ 0.0969,  0.1188,  0.2316,  ...,  0.1640, -0.0184,  0.0317]],

        [[-0.5880, -0.4821, -0.3816,  ..., -0.4197, -0.6624, -0.5947],
         [-0.3126, -0.1834, -0.0552,  ..., -0

In [10]:
y

tensor([[[-0.1274, -0.0532,  0.0979,  ..., -0.2319, -0.2353, -0.1108],
         [ 0.0930,  0.1171,  0.1863,  ..., -0.0397, -0.1367, -0.0668],
         [-0.1749, -0.0778,  0.0187,  ..., -0.1052, -0.3356, -0.2784],
         ...,
         [-0.4358, -0.4557, -0.2439,  ...,  0.5067, -0.3822, -0.3950],
         [ 0.3605,  0.3348,  0.4683,  ..., -0.1664,  0.6050,  0.3212],
         [-0.0737, -0.0730,  0.0271,  ...,  0.0052, -0.4156, -0.2450]],

        [[-0.0561,  0.0278, -0.0226,  ..., -0.2209, -0.1913, -0.1662],
         [-0.4358, -0.4557, -0.2439,  ...,  0.5067, -0.3822, -0.3950],
         [ 0.3605,  0.3348,  0.4683,  ..., -0.1664,  0.6050,  0.3212],
         ...,
         [ 0.1893,  0.2064,  0.3742,  ..., -0.0960,  0.2157,  0.1479],
         [ 0.0969,  0.1188,  0.2316,  ...,  0.1640, -0.0184,  0.0317],
         [ 0.1906,  0.2640,  0.3258,  ...,  0.3035,  0.1725,  0.2046]],

        [[-0.3126, -0.1834, -0.0552,  ..., -0.3991, -0.4640, -0.3677],
         [-0.3038, -0.2749, -0.2219,  ..., -0